## Let's use sigpy

In [ ]:
# ! conda install -c frankong sigpy # didn't work so I just pip-installed sigpy

In [1]:
# ! salloc.tinygpu --gres=gpu:1 --time=12:00:00
# ! conda activate myenv
from sigpy.mri.app import EspiritCalib as Espirit
import numpy as np
import sigpy as sp
import h5py as h5

 Args:
 
        * ksp (array): k-space array of shape [num_coils, n_ndim, ..., n_1]

        * calib (tuple of ints): length-2 image shape.

        * thresh (float): threshold for the calibration matrix.

        * kernel_width (int): kernel width for the calibration matrix.

        * max_power_iter (int): maximum number of power iterations.

        * device (Device): computing device.
        
        * crop (int): cropping threshold.


In [9]:
orgFilename = 'file_brain_AXFLAIR_201_6002903.h5'

f = h5.File('train/'+ orgFilename, 'r')
ksp = f['kspace'] # slices, coils, h, w
#ksp = np.transpose(ksp, (0,2,3,1)) 
ksp = np.swapaxes(ksp, 0,1) # -> coils, slices, h, w

In [10]:
#csm = Espirit(ksp, device=sp.Device(0)).run() # CuPy needed
scm = []
for s in range(ksp.shape[1]):
    k1 = ksp[:, s, ...]

    #c1 = Espirit(k1, device=sp.Device(0)).run() # CuPy needed
    c1 = Espirit(k1).run() # CuPy needed

    scm.append(c1)

scm = np.array(scm)

EspiritCalib:   0%|          | 0/100 [00:00<?, ?it/s]

EspiritCalib:   0%|          | 0/100 [00:00<?, ?it/s]

EspiritCalib:   0%|          | 0/100 [00:00<?, ?it/s]

EspiritCalib:   0%|          | 0/100 [00:00<?, ?it/s]

EspiritCalib:   0%|          | 0/100 [00:00<?, ?it/s]

...

EspiritCalib:   0%|          | 0/100 [00:00<?, ?it/s]

EspiritCalib:   0%|          | 0/100 [00:00<?, ?it/s]

EspiritCalib:   0%|          | 0/100 [00:00<?, ?it/s]

: 

: 

In [5]:
def saveCSM(csm: np.ndarray, name: str, mode = 'train'):
    with h5.File(name, 'w') as target: # creating a new file containing the necesary data to use it with MoDL
            if mode == 'train':
                #target.create_dataset('atb', data = atb_data)    # is the aliased/noisy image # It is not necessary for Brain_data
                target.create_dataset('trnCsm', data = csm)    # saves the coil sensitivity maps
            elif mode == 'test':
                #target.create_dataset('atb', data = atb_data)    # is the aliased/noisy image # It is not necessary for Brain_data
                target.create_dataset('tstCsm', data = csm)    # saves the coil sensitivity maps

In [7]:
filename = 'csm_' + orgFilename
saveCSM(scm, filename)

In [ ]:
import matplotlib.pyplot as plt
plt.show()

In [ ]:
def save(ground_truth: np.ndarray, csm: np.ndarray, mask: np.ndarray, name: str, mode = 'train'):
    with h5.File(name, 'w') as target: # creating a new file containing the necesary data to use it with MoDL
            # order: <KeysViewHDF5 ['atb', 'csm', 'mask', 'org']>
            if mode == 'train':
                #target.create_dataset('atb', data = atb_data)    # is the aliased/noisy image # It is not necessary for Brain_data
                target.create_dataset('trnCsm', data = csm)    # saves the coil sensitivity maps
                target.create_dataset('trnMask', data = mask)   # is the undersampling mask 
                target.create_dataset('trnOrg', data = ground_truth)#f['reconstruction_rss'])    # this is the original ground truth
            elif mode == 'test':
                #target.create_dataset('atb', data = atb_data)    # is the aliased/noisy image # It is not necessary for Brain_data
                target.create_dataset('tstCsm', data = csm)    # saves the coil sensitivity maps
                target.create_dataset('tstMask', data = mask)   # is the undersampling mask 
                target.create_dataset('tstOrg', data = ground_truth)#f['reconstruction_rss'])    # this is the original ground truth